In [37]:
import torch
import torchvision.transforms as transforms
import torchvision.ops as ops
from net import Net
import cv2
import warnings
from sliding_window import sliding_window
from image_pyramid import image_pyramid
import torch.nn.functional as F

#suppress warning
warnings.filterwarnings('ignore')

In [38]:
face_detection_net = Net()
face_detection_net.load_state_dict(torch.load("./saved_model.pth"))

<All keys matched successfully>

In [39]:
# initialize variables used for the object detection procedure
PYR_SCALE = 1.5
WINDOW_STEP = 16
ROI_SIZE = (128,128)
INPUT_SIZE = (36,36)

In [40]:
def resize_image(image, max_size):
    height, width = image.shape[:2]
    if height > width:
        new_width = width
        new_height = int(height*max_size/width)
    else:
        new_height = height
        new_width = int(width*max_size/height)

    image = cv2.resize(image, (new_width, new_height))
    return image

In [41]:
transform = transforms.Compose([ 
    transforms.ToPILImage(),
    transforms.ToTensor(),    
    transforms.Resize(INPUT_SIZE)
])

In [42]:
rois = []
locs = []

#image_path = "./image_face_detection/0000bee39176697a.jpg"
image_path = './face_detection_images/' + 'detection_test2.jpg'
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

#img = resize_image(img, 720)

(H, W) = img.shape[:2]

pyramid = image_pyramid(img, scale=PYR_SCALE, min_size=ROI_SIZE)

for image in pyramid:
    # determine the scale factor between the *original* image
    # dimensions and the *current* layer of the pyramid
    scale = W / float(image.shape[1])
    
    # for each layer of the image pyramid, loop over the sliding
    # window locations
    for (x, y, roiOrig) in sliding_window(image, WINDOW_STEP, ROI_SIZE):
        # scale the (x, y)-coordinates of the ROI with respect to the
        # *original* image dimensions
        x = int(x * scale)
        y = int(y * scale)
        w = int(ROI_SIZE[0] * scale)
        h = int(ROI_SIZE[1] * scale)
        # take the ROI and preprocess it so we can later classify the region 

        roi_tensor_gray = transform(roiOrig)

        # update our list of ROIs and associated coordinates
        rois.append(roi_tensor_gray)
        locs.append((x, y, x + w, y + h))


In [43]:
stacked_tensor = torch.stack(rois, dim=0)

print(stacked_tensor.size())

output = face_detection_net(stacked_tensor)

torch.Size([7560, 1, 36, 36])


In [44]:
probs = F.softmax(output, dim=1)
probs_list = probs.tolist()

labels = {'valid_probs': [],
          'boxes': []}

In [45]:
for i in range(0,len(probs_list)):
    if probs_list[i][1] >= 0.95:
        box = locs[i]

        labels['valid_probs'].append(probs_list[i][1])
        labels['boxes'].append(box)

tensor_boxes = torch.Tensor(labels['boxes'])
print(tensor_boxes)
tensor_probs = torch.Tensor(labels['valid_probs'])

valid_box = ops.nms(tensor_boxes, tensor_probs, iou_threshold=0.1)
        

tensor([[ 704.,  240.,  832.,  368.],
        [ 704.,  256.,  832.,  384.],
        [ 720.,  256.,  848.,  384.],
        [ 688.,  272.,  816.,  400.],
        [ 704.,  272.,  832.,  400.],
        [ 704.,  288.,  832.,  416.],
        [ 720.,  288.,  848.,  416.],
        [ 736.,  288.,  864.,  416.],
        [ 624.,  304.,  752.,  432.],
        [ 640.,  304.,  768.,  432.],
        [ 656.,  304.,  784.,  432.],
        [ 672.,  304.,  800.,  432.],
        [ 688.,  304.,  816.,  432.],
        [ 704.,  304.,  832.,  432.],
        [ 720.,  304.,  848.,  432.],
        [ 736.,  304.,  864.,  432.],
        [1040.,  304., 1168.,  432.],
        [1056.,  304., 1184.,  432.],
        [  16.,  320.,  144.,  448.],
        [ 624.,  320.,  752.,  448.],
        [ 640.,  320.,  768.,  448.],
        [ 656.,  320.,  784.,  448.],
        [ 672.,  320.,  800.,  448.],
        [ 688.,  320.,  816.,  448.],
        [ 704.,  320.,  832.,  448.],
        [ 720.,  320.,  848.,  448.],
        [  1

In [46]:
print(labels['boxes'][1])

print(valid_box)

(704, 256, 832, 384)
tensor([85, 45, 60, 56, 16, 71, 97, 95])


In [47]:
 
# Blue color in BGR 
color = (255, 0, 0) 
  
# Line thickness of 1 px 
thickness = 1

img_color = cv2.imread(image_path)
  
# Using cv2.rectangle() method 
# Draw a rectangle with blue line borders of thickness of 1 px 
for index in valid_box:
    box = labels['boxes'][index]
    (x,y,z,t) = box
    cv2.rectangle(img_color, (x,y), (z,t), color, thickness)

cv2.imshow('image', img_color)

# add wait key. window waits until user presses a key
cv2.waitKey(0)
# and finally destroy/close all open windows
cv2.destroyAllWindows()